In [1]:
# import lots of stuff
import sys
import os
import re
import tweepy
from tweepy import OAuthHandler

import numpy as np
import pandas as pd

from os import path

import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

!pip install sentence_transformers
!pip install tweet-preprocessor
!pip install unidecode
!python -m nltk.downloader all

from sentence_transformers import SentenceTransformer
import preprocessor as prep

import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 71kB 2.4MB/s 
     |████████████████████████████████| 1.1MB 10.4MB/s 
     |████████████████████████████████| 1.1MB 26.2MB/s 
     |████████████████████████████████| 3.0MB 47.7MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=883aaa208afb23fa0d09b35d0c420b6ccc3d965a384d33c1cb8b763a3ad22f06
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=01360b17e512bd89b9f72619a8368aab4aaffd93be805f66e096065cef74f48f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
     |████████████████████████████████| 245kB 3.3MB/s 
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.dow

In [2]:
#@title Enter Twitter Credentials
TWITTER_KEY = 'rxZKr5xZ9S1b6bG4jIVXVkZqu' #@param {type:"string"}
TWITTER_SECRET_KEY = 'fX1wkeXC9x7y9TcrZyBZx9b6LbVjh0500geu81ysMKpNSDkW2k' #@param {type:"string"}

In [3]:
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [4]:
max_tweets = 1000

tweet_list=[]
for tweet in tweepy.Cursor(api.user_timeline, id="balajis", include_rts=False, tweet_mode="extended").items(max_tweets):

    tweet_list.append([ 
                      tweet.id, tweet.created_at.date(), tweet.user.screen_name, tweet.user.name, tweet.user.id, tweet.favorite_count, tweet.retweet_count, tweet.full_text
                      ])

print("Downloaded {0} tweets".format(len(tweet_list)))

Downloaded 1000 tweets


In [5]:
pd.set_option('display.max_colwidth', -1)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_list, columns=['tweet_id', 'tweet_date', 'username', 'name', 'user_id', 'like_count', 'retweet_count', 'text'])
tweet_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_id,tweet_date,username,name,user_id,like_count,retweet_count,text
0,1322667920787349504,2020-10-31,balajis,balajis.com,2178012643,18,0,"@zackkanter Best example of this are the tools that are priced at roughly one headcount, like Looker."
1,1322646811740237824,2020-10-31,balajis,balajis.com,2178012643,71,10,“David Burt...is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank.”\n\nBermuda: a new remote work destination for crypto people? https://t.co/nOiHpWn8JL
2,1322640786295435264,2020-10-31,balajis,balajis.com,2178012643,153,14,"In the best case scenario, all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything.\n\nDecentralized cryptographic verification of information supply chain, starting with...Verizon?!? https://t.co/p2qcZXjqNs"
3,1322635484812619776,2020-10-31,balajis,balajis.com,2178012643,8,1,@noUpside It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table.\n\nThe whole value prop of an immutable archive is “we will not edit”. This is why chain-of-custody is a thing. Because motivated tampering is a thing.
4,1322634491081322496,2020-10-31,balajis,balajis.com,2178012643,10,0,@aashaymody @noUpside You have an untouched archive and provide commentary offsite.\n\nThere are literally millions of commentary feeds to “add color” and only a few archival sites. Sad they decided to destroy trust in their institution. Without cryptographic validation you can no longer trust the site.


In [6]:
#Create a column for hashtags
tweet_df.insert(loc=7, column='hashtag', value=tweet_df['text'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*', x)))
tweet_df.head(10)

,tweet_id,tweet_date,username,name,user_id,like_count,retweet_count,hashtag,text
0,1322667920787349504,2020-10-31,balajis,balajis.com,2178012643,18,0,[],"@zackkanter Best example of this are the tools that are priced at roughly one headcount, like Looker."
1,1322646811740237824,2020-10-31,balajis,balajis.com,2178012643,71,10,[],“David Burt...is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank.”\n\nBermuda: a new remote work destination for crypto people? https://t.co/nOiHpWn8JL
2,1322640786295435264,2020-10-31,balajis,balajis.com,2178012643,153,14,[],"In the best case scenario, all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything.\n\nDecentralized cryptographic verification of information supply chain, starting with...Verizon?!? https://t.co/p2qcZXjqNs"
3,1322635484812619776,2020-10-31,balajis,balajis.com,2178012643,8,1,[],@noUpside It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table.\n\nThe whole value prop of an immutable archive is “we will not edit”. This is why chain-of-custody is a thing. Because motivated tampering is a thing.
4,1322634491081322496,2020-10-31,balajis,balajis.com,2178012643,10,0,[],@aashaymody @noUpside You have an untouched archive and provide commentary offsite.\n\nThere are literally millions of commentary feeds to “add color” and only a few archival sites. Sad they decided to destroy trust in their institution. Without cryptographic validation you can no longer trust the site.
5,1322631666217570304,2020-10-31,balajis,balajis.com,2178012643,6,0,[],"@noUpside In both cases, the entire value proposition is about trust. Trust that it wasn’t maliciously tampered with.\n\nThe fact that the @internetarchive has publicly signaled that they CAN edit the archives means trust in their stewardship has evaporated. Fortunately we have crypto."
6,1322630495369261056,2020-10-31,balajis,balajis.com,2178012643,5,0,[],"@noUpside A waiter spits on the ground next to your food. Stares at you. Sure, you didn’t see him spit *in* your food...\n\nAn immutable archive edits the banner right above your link. It’s staring at you. Sure, you didn’t see them *edit* the content itself..."
7,1322617521283870720,2020-10-31,balajis,balajis.com,2178012643,347,47,[],"More 1984 every day.\n\nThankfully we have decentralized storage systems coming online just as the older systems get Orwellian.\n\nNot just blockchains, but IPFS, CRDTs, https://t.co/ptr4ETVkUe, and other dweb tech. Plus older but still important tools like git, BitTorrent, etc. https://t.co/ZaeSd1TFor"
8,1322615401373605888,2020-10-31,balajis,balajis.com,2178012643,1,0,[],"@rlacombe @wttj I did note the presence of proper nouns like Git and Elixir in the headlines, so they probably want to SEO for candidates with those skills.\n\nBut the sections I looked at were actually really high quality writing, not slapdash stuff. Hopefully this pays for itself."
9,1322614593701597184,2020-10-31,balajis,balajis.com,2178012643,1,0,[],"@atroyn Well, that’s an imperative now. You have to become a tech co to survive.\n\nI think the issue is that “technology” is not identical to the “tech industry”. Some people love money too much, others hate it too much. \n\nBut money is not our highest value. Math is the true core of CS."


In [7]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [8]:
stopset = stopwords.words('english')
punctuations = '''!()-=—!→–[]|{};:+`"“”\,<>/@#$%^&*_~'''

prep.set_options(prep.OPT.URL, prep.OPT.MENTION)

cleaned = [prep.clean(text) for text in tweet_df['text']]

print(cleaned)

tweet_df['cleaned'] = cleaned

#tweet_df['cleaned'] = tweet_df.cleaned.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stopset))

tweet_df['cleaned'] = tweet_df.cleaned.apply(lambda x: x.translate(str.maketrans(punctuations, ' '*len(punctuations))))
tweet_df['cleaned'] = tweet_df.cleaned.apply(lambda x: remove_emojis(x))
tweet_df['cleaned'] = tweet_df.cleaned.apply(lambda x: x.replace('...',' '))
tweet_df['cleaned'] = tweet_df.cleaned.apply(lambda x: ' '.join(x.split()))


tweet_df.head(10)

['Best example of this are the tools that are priced at roughly one headcount, like Looker.', '“David Burt...is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank.” Bermuda: a new remote work destination for crypto people?', 'In the best case scenario, all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything. Decentralized cryptographic verification of information supply chain, starting with...Verizon?!?', 'It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table. The whole value prop of an immutable archive is “we will not edit”. This is why chain-of-custody is a thing. Because motivated tampering is a thing.', 'You have an untouched archive and provide commentary offsite. There are literally millions of commentary feeds to “add color” and only a few archival sites. Sa

,tweet_id,tweet_date,username,name,user_id,like_count,retweet_count,hashtag,text,cleaned
0,1322667920787349504,2020-10-31,balajis,balajis.com,2178012643,18,0,[],"@zackkanter Best example of this are the tools that are priced at roughly one headcount, like Looker.",Best example of this are the tools that are priced at roughly one headcount like Looker.
1,1322646811740237824,2020-10-31,balajis,balajis.com,2178012643,71,10,[],“David Burt...is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank.”\n\nBermuda: a new remote work destination for crypto people? https://t.co/nOiHpWn8JL,David Burt is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank. Bermuda a new remote work destination for crypto people?
2,1322640786295435264,2020-10-31,balajis,balajis.com,2178012643,153,14,[],"In the best case scenario, all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything.\n\nDecentralized cryptographic verification of information supply chain, starting with...Verizon?!? https://t.co/p2qcZXjqNs",In the best case scenario all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything. Decentralized cryptographic verification of information supply chain starting with Verizon? ?
3,1322635484812619776,2020-10-31,balajis,balajis.com,2178012643,8,1,[],@noUpside It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table.\n\nThe whole value prop of an immutable archive is “we will not edit”. This is why chain-of-custody is a thing. Because motivated tampering is a thing.,It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table. The whole value prop of an immutable archive is we will not edit . This is why chain of custody is a thing. Because motivated tampering is a thing.
4,1322634491081322496,2020-10-31,balajis,balajis.com,2178012643,10,0,[],@aashaymody @noUpside You have an untouched archive and provide commentary offsite.\n\nThere are literally millions of commentary feeds to “add color” and only a few archival sites. Sad they decided to destroy trust in their institution. Without cryptographic validation you can no longer trust the site.,You have an untouched archive and provide commentary offsite. There are literally millions of commentary feeds to add color and only a few archival sites. Sad they decided to destroy trust in their institution. Without cryptographic validation you can no longer trust the site.
5,1322631666217570304,2020-10-31,balajis,balajis.com,2178012643,6,0,[],"@noUpside In both cases, the entire value proposition is about trust. Trust that it wasn’t maliciously tampered with.\n\nThe fact that the @internetarchive has publicly signaled that they CAN edit the archives means trust in their stewardship has evaporated. Fortunately we have crypto.",In both cases the entire value proposition is about trust. Trust that it wasn’t maliciously tampered with. The fact that the has publicly signaled that they CAN edit the archives means trust in their stewardship has evaporated. Fortunately we have crypto.
6,1322630495369261056,2020-10-31,balajis,balajis.com,2178012643,5,0,[],"@noUpside A waiter spits on the ground next to your food. Stares at you. Sure, you didn’t see him spit *in* your food...\n\nAn immutable archive edits the banner right above your link. It’s staring at you. Sure, you didn’t see them *edit* the content itself...",A waiter spits on the ground next to your food. Stares at you. Sure you didn’t see him spit in your food An immutable archive edits the banner right above your link. It’s staring at you. Sure you didn’t see them edit the content itself
7,1322617521283870720,2020-10-31,balajis,balajis.com,2

In [9]:
tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()
punctuations = '''!0123456789()-=—!→–[]{};:+`'"“”\,<>.?/@#$%^&*_~'''
stopset = stopwords.words('english')

abbrevs = {
    'dm': 'direct message',
    'ai':'artificial intelligence',
    'ar':'augmented reality',
    'vr':'virtual reality',
    'ml':'machine learning',
    'btc':'bitcoin',
    'eth':'ethereum'
}

def lemmatize(token):
    lemmatized_token1=lemmatizer.lemmatize(token, pos="v")
    lemmatized_token2=lemmatizer.lemmatize(lemmatized_token1, pos="n")
    lemmatized_token3=lemmatizer.lemmatize(lemmatized_token2, pos="a")
    lemmatized_token=lemmatizer.lemmatize(lemmatized_token3, pos="r")
    return lemmatized_token

def pre_process(corpus):
    corpus = corpus.lower()
    corpus = corpus.translate(str.maketrans(punctuations, ' '*len(punctuations)))
    tokens = tokenizer.tokenize(corpus)
    cleaned_corpus=[]
    for token in tokens:
      token = ''.join([i for i in token if not i.isdigit()])
      if ((token not in stopset) and (len(token)>2)):
        token = lemmatize(token)
        token = token.translate(str.maketrans('', '', string.punctuation))
        token = unidecode(token)
        cleaned_corpus.append(token)
      else:
        continue
    return cleaned_corpus

pre_processed = [pre_process(tweet) for tweet in cleaned]

print(pre_processed)

[['best', 'example', 'tool', 'price', 'roughly', 'one', 'headcount', 'like', 'looker'], ['david', 'burt', 'spearhead', 'project', 'use', 'island', 'test', 'grind', 'stablecoins', 'launch', 'communally', 'own', 'national', 'digital', 'bank', 'bermuda', 'new', 'remote', 'work', 'destination', 'crypto', 'people'], ['best', 'case', 'scenario', 'company', 'burn', 'trust', 'burst', 'censorious', 'madness', 'find', 'regain', 'go', 'crypto', 'everything', 'decentralize', 'cryptographic', 'verification', 'information', 'supply', 'chain', 'start', 'verizon'], ['brinkmanship', 'kill', 'trust', 'institution', 'waiter', 'openly', 'spit', 'grind', 'right', 'next', 'table', 'whole', 'value', 'prop', 'immutable', 'archive', 'edit', 'chain', 'custody', 'thing', 'motivate', 'tamper', 'thing'], ['untouched', 'archive', 'provide', 'commentary', 'offsite', 'literally', 'million', 'commentary', 'feed', 'add', 'color', 'archival', 'sit', 'sad', 'decide', 'destroy', 'trust', 'institution', 'without', 'cryptog

In [10]:
#Add cleaned text as new column
#punctuations = '''0123456789!()-=![]{};:.?+`'“”"\,<>/@#$%^&*_~'''
#cleaned = [text.lower().translate(str.maketrans(punctuations, ' '*len(punctuations))) for text in cleaned]

#tweet_df['cleaned'] = cleaned
tweet_df['tokens'] = pre_processed

#tweet_new = tweet_df[tweet_df['cleaned'].str.split().str.len()>1]

tweet_df.head(10)

,tweet_id,tweet_date,username,name,user_id,like_count,retweet_count,hashtag,text,cleaned,tokens
0,1322667920787349504,2020-10-31,balajis,balajis.com,2178012643,18,0,[],"@zackkanter Best example of this are the tools that are priced at roughly one headcount, like Looker.",Best example of this are the tools that are priced at roughly one headcount like Looker.,"[best, example, tool, price, roughly, one, headcount, like, looker]"
1,1322646811740237824,2020-10-31,balajis,balajis.com,2178012643,71,10,[],“David Burt...is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank.”\n\nBermuda: a new remote work destination for crypto people? https://t.co/nOiHpWn8JL,David Burt is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank. Bermuda a new remote work destination for crypto people?,"[david, burt, spearhead, project, use, island, test, grind, stablecoins, launch, communally, own, national, digital, bank, bermuda, new, remote, work, destination, crypto, people]"
2,1322640786295435264,2020-10-31,balajis,balajis.com,2178012643,153,14,[],"In the best case scenario, all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything.\n\nDecentralized cryptographic verification of information supply chain, starting with...Verizon?!? https://t.co/p2qcZXjqNs",In the best case scenario all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything. Decentralized cryptographic verification of information supply chain starting with Verizon? ?,"[best, case, scenario, company, burn, trust, burst, censorious, madness, find, regain, go, crypto, everything, decentralize, cryptographic, verification, information, supply, chain, start, verizon]"
3,1322635484812619776,2020-10-31,balajis,balajis.com,2178012643,8,1,[],@noUpside It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table.\n\nThe whole value prop of an immutable archive is “we will not edit”. This is why chain-of-custody is a thing. Because motivated tampering is a thing.,It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table. The whole value prop of an immutable archive is we will not edit . This is why chain of custody is a thing. Because motivated tampering is a thing.,"[brinkmanship, kill, trust, institution, waiter, openly, spit, grind, right, next, table, whole, value, prop, immutable, archive, edit, chain, custody, thing, motivate, tamper, thing]"
4,1322634491081322496,2020-10-31,balajis,balajis.com,2178012643,10,0,[],@aashaymody @noUpside You have an untouched archive and provide commentary offsite.\n\nThere are literally millions of commentary feeds to “add color” and only a few archival sites. Sad they decided to destroy trust in their institution. Without cryptographic validation you can no longer trust the site.,You have an untouched archive and provide commentary offsite. There are literally millions of commentary feeds to add color and only a few archival sites. Sad they decided to destroy trust in their institution. Without cryptographic validation you can no longer trust the site.,"[untouched, archive, provide, commentary, offsite, literally, million, commentary, feed, add, color, archival, sit, sad, decide, destroy, trust, institution, without, cryptographic, validation, long, trust, site]"
5,1322631666217570304,2020-10-31,balajis,balajis.com,2178012643,6,0,[],"@noUpside In both cases, the entire value proposition is about trust. Trust that it wasn’t maliciously tampered with.\n\nThe fact that the @internetarchive has publicly signaled that they CAN edit the archives means trust in their stewardship has evaporated. Fortunately we have crypto.",In both cases the en

In [11]:
docs = []
doc = ""
words = ""

cleaned = tweet_df['cleaned'].tolist()
tokens = [item for sublist in pre_processed for item in sublist]
words = " ".join(tokens)

doc = " ".join(cleaned)

docs.append([tweet_list[0][4], tweet_list[0][2], doc, words])

print(docs)

[[2178012643, 'balajis', "Best example of this are the tools that are priced at roughly one headcount like Looker. David Burt is spearheading projects to use the island as a testing ground for stablecoins and to launch a communally owned national digital bank. Bermuda a new remote work destination for crypto people? In the best case scenario all the companies that burnt trust in a burst of censorious madness find they can only regain it by going crypto on everything. Decentralized cryptographic verification of information supply chain starting with Verizon? ? It’s brinkmanship. It kills trust in their institution. It’s the waiter openly spitting on the ground right next to your table. The whole value prop of an immutable archive is we will not edit . This is why chain of custody is a thing. Because motivated tampering is a thing. You have an untouched archive and provide commentary offsite. There are literally millions of commentary feeds to add color and only a few archival sites. Sad

In [12]:
doc_df = pd.DataFrame(docs, columns=["userid","username","tweets","words"])

doc_df.head(5)

userid  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [17]:
tweet_df.to_csv('tweets.csv')
doc_df.to_csv('doc.csv')
!cp tweets.csv "/content/gdrive/My Drive/"
!cp doc.csv "/content/gdrive/My Drive/"

In [14]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([doc])

print(embeddings)

tf.Tensor(
[[-0.04690128 -0.04697534 -0.04685867  0.04685309  0.04647566 -0.04492144
  -0.04697491  0.0460497   0.0469752  -0.04693952 -0.03883155  0.04671131
   0.04697534  0.04697062 -0.01696596  0.04697016  0.04694313 -0.04262428
   0.04697413 -0.04695876 -0.04697534  0.04697534  0.04696505  0.04697532
   0.02184161  0.04697534  0.04163188  0.04637095 -0.04649753 -0.04697531
   0.04697534 -0.04693651  0.02715969 -0.04697492  0.04671936  0.04685365
  -0.04697534  0.04697534 -0.04697532 -0.04697534  0.04694807 -0.04590806
   0.04504065 -0.04697137  0.04697474 -0.04561432  0.04672601 -0.04427432
  -0.04695657 -0.02739551 -0.04697534  0.04697518 -0.04674091 -0.04697492
  -0.04697531 -0.04697531 -0.04697534  0.04695284  0.04697534 -0.03192039
   0.04694948 -0.0469741  -0.02046033 -0.04697534 -0.04697409 -0.04697484
   0.04697534  0.04697534 -0.00801115  0.04694803 -0.04697534  0.04697534
  -0.02760096  0.0469438   0.04697534 -0.04697498 -0.04022469  0.02546824
   0.03256538  0.04697534 -